## 데이터 로딩

In [52]:
import pandas as pd
import numpy as np

In [53]:
cafe_data = pd.read_csv("cafe_data(similarity).CSV", encoding='cp949')

In [54]:
cafe_data

,cafe_name,overview,concept,reviewer
0,"마을(보드게임 만화 카페, 궁동)",만화도 보고 보드게임도 하고 연인이나 친구들과 즐겨도 되고 혼자 공부하기도 좋네요,대화하기좋아요 좌석이편해요 친절해요 음료가맛있어요 집중하기좋아요,리뷰 260개 / 5.0 / 혼자 공부하기 좋네요
1,스토랑트커피 대전봉명점(봉명동),방음 잘 되어있음 다른데 주차해서 주차는 모르겠음,커피가맛있어요 매장이청결해요 집중하기좋아요 인테리어가멋져요 디저트가맛있어요,리뷰 112개 / 5.0 / 방음
2,탐앤탐스 대전유성점(봉명동),조용하고2층 한적해서 좋아요,커피가맛있어요 집중하기좋아요 음료가맛있어요 매장이청결해요 대화하기좋아요,리뷰 273개 / 5.0 / 조용
3,드롭탑 유성복합터미널DT점(구암동),주차가 정말 편하고 좌석이 넓습니다,커피가맛있어요 친절해요 주차하기편해요 매장이청결해요 음료가맛있어요,리뷰 84개 / 5.0 / 좌석이 넓다


In [55]:
cafe_data.drop(["reviewer"], axis=1)

,cafe_name,overview,concept
0,"마을(보드게임 만화 카페, 궁동)",만화도 보고 보드게임도 하고 연인이나 친구들과 즐겨도 되고 혼자 공부하기도 좋네요,대화하기좋아요 좌석이편해요 친절해요 음료가맛있어요 집중하기좋아요
1,스토랑트커피 대전봉명점(봉명동),방음 잘 되어있음 다른데 주차해서 주차는 모르겠음,커피가맛있어요 매장이청결해요 집중하기좋아요 인테리어가멋져요 디저트가맛있어요
2,탐앤탐스 대전유성점(봉명동),조용하고2층 한적해서 좋아요,커피가맛있어요 집중하기좋아요 음료가맛있어요 매장이청결해요 대화하기좋아요
3,드롭탑 유성복합터미널DT점(구암동),주차가 정말 편하고 좌석이 넓습니다,커피가맛있어요 친절해요 주차하기편해요 매장이청결해요 음료가맛있어요


## 세 텍스트 필드들을 모아서 텍스트 유사도에 사용할 텍스트 필드 하나를 생성

In [57]:
for f in ['cafe_name','overview','concept']:
  cafe_data[f] = cafe_data[f].fillna('')

In [58]:
def combine_features(row):
    try:
        return row['cafe_name']+" "+row['overview']+" "+row["concept"] # 세 개의 필드(열)를 붙임
    except:
        print ("Error:", row)

In [59]:
cafe_data["combined_features"] = cafe_data.apply(combine_features,axis=1)
cafe_data = cafe_data.reset_index() # 매우 중요한 부분 (찾아볼것!)

In [60]:
cafe_data["combined_features"]

0    마을(보드게임 만화 카페, 궁동) 만화도 보고 보드게임도 하고 연인이나 친구들과 즐...
1    스토랑트커피 대전봉명점(봉명동) 방음 잘 되어있음 다른데 주차해서 주차는 모르겠음 ...
2    탐앤탐스 대전유성점(봉명동) 조용하고2층 한적해서 좋아요 커피가맛있어요 집중하기좋아...
3    드롭탑 유성복합터미널DT점(구암동) 주차가 정말 편하고 좌석이 넓습니다 커피가맛있어...
Name: combined_features, dtype: object

## TF-IDF 기반 벡터 생성 후 코사인 유사도로 영화들간의 유사도 계산

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [62]:
tfidfvectorizer = TfidfVectorizer(analyzer='word', norm='l2')

In [63]:
tfidf_matrix = tfidfvectorizer.fit_transform(cafe_data["combined_features"])

In [64]:
tfidf_matrix.shape

(4, 48)

In [65]:
cosine_sim = cosine_similarity(tfidf_matrix)

In [66]:
df_cosine_sim = pd.DataFrame(data = cosine_sim)
df_cosine_sim.head()

,0,1,2,3
0,1.000000,0.027117,0.117267,0.071579
1,0.027117,1.000000,0.190945,0.071899
2,0.117267,0.190945,1.000000,0.132281
3,0.071579,0.071899,0.132281,1.000000


## 아이템 기반 추천 함수 만들기

In [67]:
def get_name_from_index(df, index):
    return df[df.index == index]["cafe_name"].values[0]

def get_index_from_name(df, name):
    return df[df.cafe_name == name]["index"].values[0]

In [68]:
cosine_sim[0]

array([1.        , 0.02711736, 0.11726688, 0.07157918])

In [69]:
for cs in enumerate(cosine_sim[0]):
  print(cs)

(0, 1.0)
(1, 0.027117357318650035)
(2, 0.1172668782003994)
(3, 0.0715791763189345)


In [72]:
def reco_top_similar_cafes(cafe_name, n=10):
  cafe_index = get_index_from_name(cafe_data, cafe_name) # get_index_from_title 함수를 따로 만들어서 사용
  similar_cafes =  enumerate(cosine_sim[cafe_index]) # movie_index에 해당하는 행의 데이터를 읽어옴 (인덱스, 유사도값)
  sorted_similar_cafes = sorted(similar_cafes, key=lambda x:x[1], reverse=True) # 유사도값을 내림차순으로 (x[1]만 쓰면 오름차순이지만, reverse=True 로 설정했기 때문에 내림차순이 됨)

  ret_cafes = []
  i = 0
  for element in sorted_similar_cafes: # 유사도 값이 내림차순으로 정렬된 리스트를 맨 위부터 하나씩 찾는다
    name = get_name_from_index(cafe_data, element[0]) 
    ret_cafes.append(name) # 영화 제목 append됨.
    i=i+1
    if i >= n: # 요청된 숫자만큼 찾는다
      break # 요청된 숫자만큼 찾는다
  return ret_cafes # 영화 제목들이 append 된 채 리턴이 된다. 밑에 예시들을 참고하자. 

In [73]:
print(reco_top_similar_cafes('마을(보드게임 만화 카페, 궁동)', 2))

['마을(보드게임 만화 카페, 궁동)', '탐앤탐스 대전유성점(봉명동)']
